In [1]:
import os
import sys


module_path = os.path.abspath(os.path.join('../../src'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

/home/enis/projects/nna/src


In [2]:
from labeling_utils import save_to_csv,cut_random_file
from fileUtils import read_file_properties,list_files,getLength
from multiprocessing.dummy import Pool as ThreadPool
import time

from IPython.display import display
import ipywidgets as widgets

import random

import glob
import subprocess


In [3]:
from pathlib import Path
# PARAMETERS
NUMBER_OF_SPLITS=20*1000
THREAD_COUNT=20

SAMPLES_DIR= Path("/scratch/enis/data/nna/labeling")
split_folder = SAMPLES_DIR / "splitsv3/"
split_length_seconds=10

flder="/home/enis/projects/nna/data/"

# files_list_path=flder+"mp3files.txt"
# fileswlen_path = flder+"mp3files_wlen.txt"

# files_list_path=flder+"flacfilesv2.txt"
# fileswlen_path = flder+"flacfiles_wlenv2.txt"

files_list_path=flder+"flacfilesv3.txt"
fileswlen_path = flder+"flacfiles_wlenv3.txt"



In [4]:
!ls '/home/enis/projects/nna/data/flacfiles_wlenv3.txt'


/home/enis/projects/nna/data/flacfiles_wlenv3.txt


In [5]:
Path(split_folder).mkdir(exist_ok=True)

In [9]:

if not Path(fileswlen_path).exists():
    print("searching given directory")
    ignore_folders=['/tank/data/nna/real/stinchcomb/']
    search_path="/tank/data/nna/real/"

    files_path_list=list_files(search_path,ignore_folders)
else:
    with open(files_list_path,"r") as f:
        lines=f.readlines()
        files_path_list=[line.strip() for line in lines]


In [6]:
from pathlib import Path
if not Path(fileswlen_path).exists():
    length_dict={}
    for f in files_path_list:
        try:
            length=float(getLength(f))
            length_dict[f]=length
        except:
            print(f)

    length_list=list(length_dict.items())
    with open(fileswlen_path,"w") as f:
        for line in length_list:
            f.write(",".join([line[0],str(line[1])])+"\n")

with open(fileswlen_path,"r") as f:
    lines=f.readlines()
    fileswlen=[line.strip().split(",") for line in lines]

In [17]:
# !conda run -n speechEnv python /home/enis/projects/nna/src/slack_message.py -m "length ended" 

In [7]:
fileswlen[0]

['/tank/data/nna/real/anwr/31/2019/S4A10297_20190504_000000.flac', '4560.0']

In [ ]:
# split_folder

In [21]:
# ([i for i in fileswlen if float(i[1])<=10])

In [7]:
fileswlen=[i for i in fileswlen if float(i[1])>10]

In [8]:
try:
    s=time.time()
    progress = widgets.FloatProgress(value=0.0, min=0.0, max=NUMBER_OF_SPLITS)
    display(progress)

    def work(index):
        input_mp3_file,length_seconds=random.choice(fileswlen)
        length_min = int(float(length_seconds)//60)

        cut_random_file(input_mp3_file,length=split_length_seconds,
                        split_folder=split_folder,total_minute=length_min-1,depth=0,backend="ffmpeg")
        progress.value+=1

    pool = ThreadPool(THREAD_COUNT)
    results = pool.map(work, range(NUMBER_OF_SPLITS))
    pool.close()
    pool.join()
except Exception as e: 
    print(e)
    !conda run -n speechEnv python /home/enis/projects/nna/src/slack_message.py -m "split failed" 

FloatProgress(value=0.0, max=20000.0)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



empty range for randrange() (0,0, 0)


In [11]:
progress

FloatProgress(value=15832.0, max=20000.0)

In [9]:
!conda run -n speechEnv python /home/enis/projects/nna/src/slack_message.py -m "split ended" 

In [22]:
# from IPython.display import display
# import ipywidgets as widgets
# progress = widgets.FloatProgress(value=0.0, min=0.0, max=NUMBER_OF_SPLITS)
# display(progress)

# progress.value+=1

# !conda run -n speechEnv python /home/enis/projects/nna/src/slack_message.py -m "split ended" 